In [2]:
import json
from dataclasses import dataclass

import cv2

instructions = json.loads(open('instruction.json').read())

In [3]:
import numpy as np
np.random.seed(0)
img = np.array(instructions['colors'])[np.array(instructions['color_idx'])].astype(np.uint8)

In [4]:
@dataclass
class Config:
    height: int = 0.19
    radius: int = 0.47
    edge_width: int = 2
    arrow_width: int = 2
    T: np.array = np.array([
        [0.71, 0.5],
        [-0.71, 0.9]
    ])
    edge_color: tuple[int] = (70, 70, 70)
    arrow_color: tuple[int] = (0, 0, 0)

In [5]:
def draw_block(img, size:int, config: Config, pos_x=0.0, pos_y=0.0, color=(0,0,255), scale=None, height_offset=0):
    if scale is None:
        scale = img.shape[0]//4
    edge_color = config.edge_color
    corners = np.array([[0,1], [0,0],[1, 0],[1,1]]) * size * 1.0 + np.array([pos_y, pos_x])
    corners -= 0.5
    coords = (config.T @ corners.T * scale + RES//2).T.astype(int) + np.array([height_offset, 0])
    lower_coords = coords[:-1,:] - np.array([config.height * 2 * scale, 0]).astype(int)
    # return coords, lower_coords
    # filling 
    outline_coords = np.concatenate((coords[[2, 3, 0], :], lower_coords), axis=0)
    cv2.fillPoly(img, pts=[outline_coords[:,::-1]], color=color)
    
    # drawing upper contours
    cv2.polylines(img,[coords.reshape(-1, 1, 2)[:,:,::-1]],True,edge_color, thickness=config.edge_width)
    
    # drawing lower contours
    cv2.polylines(img,[lower_coords.reshape(-1, 1, 2)[:,:,::-1]],False,edge_color, thickness=config.edge_width)
    
    circle_height = int(config.radius * np.abs(config.T[0, :]).mean() * scale)
    circle_width = int(config.radius * np.abs(config.T[1, :]).mean() * scale)
    
    # drawing vertical lines
    cv2.line(img, coords[0, ::-1], lower_coords[0, ::-1], edge_color, thickness=config.edge_width)
    cv2.line(img, coords[1, ::-1], lower_coords[1, ::-1], edge_color, thickness=config.edge_width)
    cv2.line(img, coords[2, ::-1], lower_coords[2, ::-1], edge_color, thickness=config.edge_width)
    for x in range(size):
        for y in range(size):
            pos = np.array([[pos_y + y, pos_x + x]])
            pos = (config.T @ pos.T * scale + RES//2).T.astype(int) + np.array([height_offset, 0])
            cv2.ellipse(img, pos[0, ::-1], (circle_width, circle_height), 
                   0, 180, 360, edge_color, thickness=config.edge_width) 
            
            cv2.ellipse(img, pos[0, ::-1] + np.array([0, config.height * scale]).astype(int), (circle_width, circle_height), 
                   0, 0, 360, edge_color, thickness=config.edge_width) 
            
            # drawing vertical lines
            cv2.line(img, pos[0, ::-1] + np.array([-circle_width, 0]).astype(int), 
                        pos[0, ::-1] + np.array([-circle_width, config.height * scale]).astype(int), edge_color, thickness=config.edge_width)
            cv2.line(img, pos[0, ::-1] + np.array([+circle_width, 0]).astype(int), 
                        pos[0, ::-1] + np.array([+circle_width, config.height * scale]).astype(int), edge_color, thickness=config.edge_width)

In [6]:
def draw_arrow(img, start, end, config: Config, scale=None, color=(0,0,0)):
    res = img.shape[0]
    if scale is None:
        scale = res//4
    start_coords = (config.T @ start[:, None] * scale + res//2).T.astype(int)[0, :] + np.array([- 2 *config.height * scale, 0]).astype(int)
    end_coords = (config.T @ end[:, None] * scale + res//2).T.astype(int)[0, :] + np.array([config.height * scale, 0]).astype(int)
    l = np.linalg.norm(end_coords - start_coords)
    cv2.arrowedLine(img, start_coords[::-1], end_coords[::-1], 
                                     color, config.arrow_width, tipLength=5/l)  

In [8]:
import pandas as pd

RES = 3000
MARGIN = 50
scale = RES // 60
plate_size = 32
NUM_PAGES = 15
min_pos = -50
max_pos = 50
r = max_pos - min_pos

In [6]:

config = Config()
arrow_test_config = Config(arrow_width=4)
blocks = []
for y, row in enumerate(instructions['color_idx']):
    for x, color_idx in enumerate(row):
        if color_idx == 4:
            continue
        blocks.append({'x': 31-x, 'y': 31-y, 'color_idx': int(color_idx), 'pos_x': 0.0, 'pos_y': 0.0, 'nx': 999, 'ny': 999})
        
blocks = pd.DataFrame(blocks)
done_blocks = []
blocks = blocks.sample(frac=1)
viss = []
arrow_test_res = RES // 4
arrow_test_scale = arrow_test_res // 60
arrow_test_overlay_threshold = 0.15
for page_idx in range(NUM_PAGES):
    
    img = np.ones((RES, RES, 3), dtype=np.uint8)*255
    arrows_img = np.zeros((arrow_test_res, arrow_test_res, 3), dtype=np.uint8)
    draw_block(img, plate_size, config, -plate_size//2, -plate_size//2, scale=scale, color=instructions['colors'][-3])
    if done_blocks:
        for _, done_block in pd.DataFrame(done_blocks).sort_values(['x', 'y'], ascending=False).iterrows():
            x = done_block['x']
            y = done_block['y']
            color_idx = done_block['color_idx']
            draw_block(img, 1, config, x-plate_size//2, y-plate_size//2, scale=scale, color=(128,128,128), height_offset=int(config.height * 2 * scale))
        
    page_blocks = blocks.loc[blocks.index.values[page_idx::NUM_PAGES]].copy()
    for i, block in page_blocks.sort_values(['x', 'y'], ascending=False).iterrows():
        x = block['x']
        y = block['y']
        color_idx = int(block['color_idx'])
        while True:
            pos = np.random.random(2) * r + min_pos
            if all(np.abs(pos) <  plate_size//2 + 1):
                continue
            # pos = np.array([32,32])
            ppos = config.T @ pos[:, None] * scale + RES // 2
            if any(ppos < MARGIN) or any(ppos > RES - MARGIN):
                continue
            distances = np.linalg.norm(page_blocks[['pos_y', 'pos_x']].values - pos, axis=-1)
            if distances.min() < 2:
                print(pos)
                continue
            page_blocks.loc[i, 'pos_y'] = pos[0]
            page_blocks.loc[i, 'pos_x'] = pos[1]
            arrow_test_img = np.zeros((arrow_test_res, arrow_test_res, 3), dtype=np.uint8)
            draw_arrow(arrow_test_img, pos, np.array([y-plate_size//2, x-plate_size//2]), arrow_test_config, scale=arrow_test_scale, color=(255,255,255))
            overlay = arrow_test_img & arrows_img
            overlay_sum = overlay.sum()
            if overlay_sum and overlay_sum/arrow_test_img.sum() > arrow_test_overlay_threshold:
                continue
            break
        
        draw_arrow(arrows_img, pos, np.array([y-plate_size//2, x-plate_size//2]), arrow_test_config, scale=arrow_test_scale, color=(255,255,255))
        draw_arrow(img, pos, np.array([y-plate_size//2, x-plate_size//2]), config, scale=scale)
        draw_block(img, 1, config, pos[1], pos[0], scale=scale, color=instructions['colors'][color_idx])
        # draw_block(img, 1, x-plate_size//2, y-plate_size//2, scale=scale, color=instructions['colors'][color_idx], height_offset=int(HEIGHT * 2 * scale))
        done_blocks.append(block)
    viss.append(img[::-1, ::-1, :])      

[-19.07487104  -9.6781967 ]
[ 13.34148215 -19.26238829]
[  3.72790254 -22.54249512]
[ -6.80062156 -21.5913865 ]
[-31.02463835   2.08747952]
[28.97696137 -3.28805928]
[-17.38823077 -19.9810542 ]
[-26.32099634  -5.16143268]
[-13.67760381 -25.32275967]
[-16.21591331 -19.67471796]
[21.26394626  2.39944797]
[-10.72474608 -34.58701483]
[19.43938887 27.82417274]
[ 24.37942986 -11.07129252]
[-18.46127959 -26.45175241]
[33.60531379  4.20173544]
[-14.31949188 -25.74679837]
[-28.12166902   5.07992867]
[-14.01259765 -32.41969808]
[ -6.34676123 -32.26628481]
[-29.136048    -5.15931833]
[-27.62822336   3.98855007]
[-24.31869632   8.50458859]
[19.66782618 13.53138986]
[-12.98047548 -34.60572019]
[ 9.20882119 32.8734302 ]
[19.08707716 25.79618092]
[29.35970421 12.91156239]
[ 3.27205785 30.92119021]
[-14.36572653 -25.9163651 ]
[21.12749725  0.67697444]
[ -3.31377323 -28.83624888]
[-11.42738895 -27.22936028]
[18.75489576 10.61718638]
[  5.28857605 -19.35730213]
[ -2.5733207  -20.56859861]
[21.65374197  

In [12]:
for i, vis in enumerate(viss):
    cv2.imwrite(f'pages/page{i+1}.png', vis)

In [15]:

config = Config()
img = np.ones((RES, RES, 3), dtype=np.uint8)*255
draw_block(img, plate_size, config, -plate_size//2, -plate_size//2, scale=scale, color=instructions['colors'][-3])
blocks = []
for y, row in enumerate(instructions['color_idx']):
    for x, color_idx in enumerate(row):
        if color_idx == 4:
            continue
        blocks.append({'x': 31-x, 'y': 31-y, 'color_idx': int(color_idx), 'pos_x': 0.0, 'pos_y': 0.0, 'nx': 999, 'ny': 999})
        
blocks = pd.DataFrame(blocks)
for i, block in blocks.sort_values(['x', 'y'], ascending=False).iterrows():
    x = block['x']
    y = block['y']
    color_idx = int(block['color_idx'])
    draw_block(img, 1, config, x-plate_size//2, y-plate_size//2, scale=scale, color=instructions['colors'][color_idx], height_offset=int(config.height * 2 * scale))
vis = img[::-1, ::-1, :]

In [16]:
cv2.imwrite(f'pages/final.png', vis)

True